# CalciumChannel

## 电生理特性

钙离子通道是一类高度重要的膜通道，负责调节钙离子在细胞膜上的跨膜流动。钙离子在神经系统中不仅参与膜电位的变化，还在以下过程中扮演着关键角色：

- 动作电位的产生与延续：某些神经元类型依赖钙通道触发或维持放电节律。
- 突触传递：钙离子内流触发神经递质释放。
- 细胞内信号转导：钙离子可调控蛋白激酶活性、基因表达、酶促反应等。
- 钙依赖通道调控：许多钾通道或混合通道也受到钙浓度的调节。

钙通道通常具有缓慢激活、钙依赖失活、以及高选择性等特点。它们常常与钙泵和缓冲蛋白共同作用，以维持钙稳态。

## 建模实现

在 `braincell` 中，钙通道的建模通过继承 `CalciumChannel` 来实现，而 `CalciumChannel` 本身又继承自 `Channel`，属于 `IonChannel` 的一部分。

因此，在实际应用中我们需要实现需要的接口，如 `current(V, Ca)`、`compute_derivative(V, Ca)`等。

其中，`Ca` 参数是一个 `IonInfo` 类型，包含通道局部的钙浓度信息。而关于 `IonInfo` 的信息，我们会在 `Ion` 部分进行详细介绍。

通过这些方法，我们可以构建具有复杂动力学特性的钙通道，比如电压和钙浓度双重调控、钙依赖失活、稳态激活门控等。

在了解了基本的建模后，我们来看一个实际实现： `ICaN_IS2008`。

这是由 Inoue & Strowbridge 在 2008 年提出的一个钙激活的非选择性阳离子通道模型。不难看出我们对离子通道有一套固定的命名规则，即通道名称_作者+文章发表年份，这样可以保证各通道的便于查找。

回到模型本身，该模型的电流由下列公式描述：

$$
\begin{aligned}
I_{\text{CAN}} &= g_{\text{max}} \cdot M([\mathrm{Ca}^{2+}]_i) \cdot p \cdot (V - E) \\\\
M([\mathrm{Ca}^{2+}]_i) &= \frac{[\mathrm{Ca}^{2+}]_i}{0.2 + [\mathrm{Ca}^{2+}]_i} \\\\
\frac{dp}{dt} &= \frac{\phi \cdot (p_\infty - p)}{\tau_p} \\\\
p_\infty &= \frac{1}{1 + \exp\left(-\frac{V + 43}{5.2}\right)} \\\\
\tau_p &= \frac{2.7}{\exp\left(-\frac{V + 55}{15}\right) + \exp\left(\frac{V + 55}{15}\right)} + 1.6
\end{aligned}
$$

其中：
- $M$ 是钙依赖激活函数
- $p$ 是电压门控激活变量
- $\phi$ 是温度因子
- $E$ 是反转电位
- $g_{\text{max}}$ 是最大电导

以下是该模型的代码实现：

```python
class ICaN_IS2008(CalciumChannel):
    def __init__(
        self,
        size,
        E=10. * u.mV,
        g_max=1. * (u.mS / u.cm ** 2),
        phi=1.,
        name=None,
    ):
        super().__init__(size=size, name=name)
        self.E = brainstate.init.param(E, self.varshape, allow_none=False)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)
        self.phi = brainstate.init.param(phi, self.varshape, allow_none=False)

    def init_state(self, V, Ca: IonInfo, batch_size: int = None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, Ca: IonInfo, batch_size=None):
        V = V.to_decimal(u.mV)
        self.p.value = 1.0 / (1 + u.math.exp(-(V + 43.) / 5.2))

    def compute_derivative(self, V, Ca: IonInfo):
        V = V.to_decimal(u.mV)
        p_inf = 1.0 / (1 + u.math.exp(-(V + 43.) / 5.2))
        tau_p = 2.7 / (u.math.exp(-(V + 55.) / 15.) + u.math.exp((V + 55.) / 15.)) + 1.6
        self.p.derivative = self.phi * (p_inf - self.p.value) / tau_p / u.ms

    def current(self, V, Ca: IonInfo):
        M = Ca.C / (Ca.C + 0.2 * u.mM)
        g = self.g_max * M * self.p.value
        return g * (self.E - V)
```

在这个例子中，我们建立了一个具体的钙离子通道的子类，它继承自 `CalciumChannel` 。观察代码发现，在面对具体通道的建模时，我们的 `CalciumChannel` 具有很强的灵活性，只需要设定好相关的参数，并且依照公式将不同的变量用代码表达出来即可。

当然，对于其他通道，得益于我们建模的统一性，我们的代码依旧保证了高度的灵活性。